# ADS-508-01-SP23 Team 8: Final Project

# Setup Database and Athena Tables

Much of the code is modified from `Fregly, C., & Barth, A. (2021). Data science on AWS: Implementing end-to-end, continuous AI and machine learning pipelines. O’Reilly.`

## Install missing dependencies

[PyAthena](https://pypi.org/project/PyAthena/) is a Python DB API 2.0 (PEP 249) compliant client for Amazon Athena.

In [2]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

## Globally import libraries

In [3]:
import boto3
from botocore.client import ClientError
import sagemaker
import pandas as pd
from pyathena import connect
from IPython.core.display import display, HTML

## Instantiate AWS SageMaker session

In [4]:
session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
def_bucket = sagemaker_session.default_bucket()
bucket = 'sagemaker-us-east-ads508-sp23-t8'

s3 = boto3.Session().client(service_name="s3",
                            region_name=region)

In [5]:
setup_s3_bucket_passed = False
ingest_create_athena_db_passed = False
ingest_create_athena_table_tsv_passed = False

In [6]:
print(f"Default bucket: {def_bucket}")
print(f"Public T8 bucket: {bucket}")

Default bucket: sagemaker-us-east-1-657724983756
Public T8 bucket: sagemaker-us-east-ads508-sp23-t8


## Verify S3 Bucket Creation

In [7]:
%%bash

aws s3 ls s3://${bucket}/

2023-03-16 17:05:02 aws-athena-query-results-657724983756-us-east-1
2023-03-02 16:56:48 sagemaker-studio-657724983756-5nh7ydsouq7
2023-03-02 17:25:41 sagemaker-studio-657724983756-7yc8bp8xk0b
2023-03-02 17:01:51 sagemaker-us-east-1-657724983756
2023-03-17 05:19:31 sagemaker-us-east-ads508-sp23-t8


In [8]:
response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print(f"[ERROR] Cannot find bucket {bucket} in {response} due to {e}.")

{'ResponseMetadata': {'RequestId': 'QZWPKFNQS33H84ZZ', 'HostId': 'W63+Lw/FFWHy8yIxI1w7Z1Ctn9EX9eVP5h9quc3mwbYAAMHMCtUrEQCh/14InDA8rqe2qXKl04c=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'W63+Lw/FFWHy8yIxI1w7Z1Ctn9EX9eVP5h9quc3mwbYAAMHMCtUrEQCh/14InDA8rqe2qXKl04c=', 'x-amz-request-id': 'QZWPKFNQS33H84ZZ', 'date': 'Sat, 25 Mar 2023 16:50:40 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


In [9]:
%store setup_s3_bucket_passed

Stored 'setup_s3_bucket_passed' (bool)


## Create Athena Database

In [10]:
database_name = "ads508_t8"

In [11]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = f"s3://{bucket}/athena/staging"
print(s3_staging_dir)

s3://sagemaker-us-east-ads508-sp23-t8/athena/staging


In [12]:
conn = connect(region_name=region,
               s3_staging_dir=s3_staging_dir)

### Verify The Database Has Been Created Succesfully

In [13]:
show_db_stmnt = "SHOW DATABASES"

df_show = pd.read_sql(show_db_stmnt,
                      conn)
df_show.head(17)

,database_name
0,ads508_t8
1,default
2,dsoaws
3,sagemaker_featurestore


In [14]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [15]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


In [36]:
cen_tsv_tbl_name = 'census'
ceb_tsv_tbl_name = 'census_block'
evi_tsv_tbl_name = 'evictions'
cri_pqt_tbl_name = 'crime_pqt'
grd_tsv_tbl_name = 'grad_outcomes'
hsi_tsv_tbl_name = 'hs_info'
job_tsv_tbl_name = 'jobs'

In [17]:
# Set S3 path to Parquet data
abt_s3_data_path = f"s3://{bucket}/ABT"

cen_tsv_tbl_name = 'census'
abt_tbl_name = 'abt'
drop_abt_tbl_stmnt = f"""DROP TABLE IF EXISTS {database_name}.{abt_tbl_name}"""

# SQL statement to execute
create_abt_tble_stmnt = f"""
CREATE TABLE IF NOT EXISTS {database_name}.{abt_tbl_name}
WITH (
    external_location = '{abt_s3_data_path}'
    )
AS
SELECT
        cen.censustract,
        cen.borough,
        cen.totalpop,
        cen.men,
        cen.women,
        cen.hispanic,
        cen.white,
        cen.black,
        cen.native,
        cen.asian,
        cen.citizen,
        cen.income,
        cen.poverty,
        cen.childpoverty,
        cen.professional,
        cen.service,
        cen.office,
        cen.construction,
        cen.production,
        cen.drive,
        cen.carpool,
        cen.transit,
        cen.walk,
        cen.othertransp,
        cen.workathome,
        cen.meancommute,
        cen.employed,
        cen.privatework,
        cen.publicwork,
        cen.selfemployed,
        cen.familywork,
        cen.unemployment,
        ceb.blockCode,
        ceb.min_lat,
        ceb.max_lat,
        ceb.min_long,
        ceb.max_long
FROM {database_name}.{cen_tsv_tbl_name} AS cen
LEFT JOIN (
    SELECT
        substr(blockCode,1,11) AS blockCode,
        count(*),
        min(latitude) AS min_lat,
        max(latitude) AS max_lat,
        min(longitude) AS min_long,
        max(longitude) AS max_long     
    FROM {database_name}.{ceb_tsv_tbl_name}
    GROUP BY substr(blockCode,1,11)
    ORDER BY count(*) DESC
    LIMIT 50000
    ) AS ceb
    ON cen.censustract = substr(ceb.blockCode,1,11)
"""

print(f'Create table statement:\n{create_abt_tble_stmnt}')

pd.read_sql(drop_abt_tbl_stmnt,
            conn)

pd.read_sql(create_abt_tble_stmnt,
            conn)

In [18]:
abt_select_to_join_stmnt01 = f"""
SELECT
    cen.censustract,
    cen.borough,
    cen.totalpop,
    cen.men,
    cen.women,
    cen.hispanic,
    cen.white,
    cen.black,
    cen.native,
    cen.asian,
    cen.citizen,
    cen.income,
    cen.poverty,
    cen.childpoverty,
    cen.professional,
    cen.service,
    cen.office,
    cen.construction,
    cen.production,
    cen.drive,
    cen.carpool,
    cen.transit,
    cen.walk,
    cen.othertransp,
    cen.workathome,
    cen.meancommute,
    cen.employed,
    cen.privatework,
    cen.publicwork,
    cen.selfemployed,
    cen.familywork,
    cen.unemployment,
    cvi.blockCode,
    cvi.eviction_count_x_lat_long
FROM {database_name}.{cen_tsv_tbl_name} AS cen
LEFT JOIN (
    SELECT
        ceb.blockCode AS blockCode,
        sum(evi.eviction_count_x_lat_long) AS eviction_count_x_lat_long
    FROM (
        SELECT
            substr(blockCode,1,11) AS blockCode,
            count(*),
            min(latitude) AS min_lat,
            max(latitude) AS max_lat,
            min(longitude) AS min_long,
            max(longitude) AS max_long     
        FROM {database_name}.{ceb_tsv_tbl_name}
        GROUP BY substr(blockCode,1,11)
        ORDER BY count(*) DESC
        ) AS ceb
    INNER JOIN (
        SELECT
            CAST(latitude AS DOUBLE) AS latitude,
            CAST(longitude AS DOUBLE) AS longitude,
            count(*) AS eviction_count_x_lat_long
        FROM {database_name}.{evi_tsv_tbl_name}
        WHERE latitude != ''
        GROUP BY latitude, longitude
        ORDER BY count(*) DESC
        ) AS evi
    ON evi.latitude >= ceb.min_lat
        AND evi.latitude <= ceb.max_lat
        AND evi.longitude >= ceb.min_long
        AND evi.longitude <= ceb.max_long
    GROUP BY ceb.blockCode
    LIMIT 50000
    ) AS cvi
    ON cen.censustract = cvi.blockCode
ORDER BY cen.censustract
"""

print(abt_select_to_join_stmnt01)

abt_select_to_join_df01 = pd.read_sql(abt_select_to_join_stmnt01,
                                      conn)
print(abt_select_to_join_df01.shape)
display(abt_select_to_join_df01.head(15))


SELECT
    cen.censustract,
    cen.borough,
    cen.totalpop,
    cen.men,
    cen.women,
    cen.hispanic,
    cen.white,
    cen.black,
    cen.native,
    cen.asian,
    cen.citizen,
    cen.income,
    cen.poverty,
    cen.childpoverty,
    cen.professional,
    cen.service,
    cen.office,
    cen.construction,
    cen.production,
    cen.drive,
    cen.carpool,
    cen.transit,
    cen.walk,
    cen.othertransp,
    cen.workathome,
    cen.meancommute,
    cen.employed,
    cen.privatework,
    cen.publicwork,
    cen.selfemployed,
    cen.familywork,
    cen.unemployment,
    cvi.blockCode,
    cvi.eviction_count_x_lat_long
FROM ads508_t8.census AS cen
LEFT JOIN (
    SELECT
        ceb.blockCode AS blockCode,
        sum(evi.eviction_count_x_lat_long) AS eviction_count_x_lat_long
    FROM (
        SELECT
            substr(blockCode,1,11) AS blockCode,
            count(*),
            min(latitude) AS min_lat,
            max(latitude) AS max_lat,
            min(longitude)

,censustract,borough,totalpop,men,women,hispanic,white,black,native,asian,...,workathome,meancommute,employed,privatework,publicwork,selfemployed,familywork,unemployment,blockCode,eviction_count_x_lat_long
0,36005000100,Bronx,7703,7133,570,29.9,6.1,60.9,0.2,1.6,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,None,NaN
1,36005000200,Bronx,5403,2659,2744,75.8,2.3,16.0,0.0,4.2,...,0.0,43.0,2308,80.8,16.2,2.9,0.0,7.7,36005000200,31.0
2,36005000400,Bronx,5915,2896,3019,62.7,3.6,30.7,0.0,0.3,...,2.1,45.0,2675,71.7,25.3,2.5,0.6,9.5,36005000400,46.0
3,36005001600,Bronx,5879,2558,3321,65.1,1.6,32.4,0.0,0.0,...,1.7,38.8,2120,75.0,21.3,3.8,0.0,8.7,36005001600,10.0
4,36005001900,Bronx,2591,1206,1385,55.4,9.0,29.0,0.0,2.1,...,6.2,45.4,1083,76.8,15.5,7.7,0.0,19.2,36005001900,230.0
5,36005002000,Bronx,8516,3301,5215,61.1,1.6,31.1,0.3,3.3,...,0.0,46.0,2508,71.0,21.3,7.7,0.0,17.2,36005002000,69.0
6,36005002300,Bronx,4774,2130,2644,62.3,0.2,36.5,1.0,0.0,...,4.1,42.7,1191,74.2,16.1,9.7,0.0,18.9,None,NaN
7,36005002400,Bronx,150,109,41,0.0,52.0,48.0,0.0,0.0,...,0.0,NaN,113,62.8,37.2,0.0,0.0,0.0,36005002400,169.0
8,36005002500,Bronx,5355,2338,3017,76.5,1.5,18.9,0.0,3.0,...,2.7,35.5,1691,85.1,8.3,6.1,0.5,9.4,36005002500,22.0
9,36005002701,Bronx,3016,1375,1641,68.0,0.0,31.2,0.0,0.0,...,1.6,42.8,1102,86.9,8.5,4.5,0.0,15.2,None,NaN



SELECT
    ceb.blockCode,
    evi.year,
    sum(evi.eviction_count_x_lat_long) AS annual_evictions_x_census_tract
FROM (SELECT
    substr(blockCode,1,11) AS blockCode,
    count(*),
    min(latitude) AS min_lat,
    max(latitude) AS max_lat,
    min(longitude) AS min_long,
    max(longitude) AS max_long     
FROM ads508_t8.census_block
GROUP BY substr(blockCode,1,11)
ORDER BY count(*) DESC) AS ceb
INNER JOIN (
SELECT
    CAST(latitude AS DOUBLE) AS latitude,
    CAST(longitude AS DOUBLE) AS longitude,
    CAST(YEAR(date_parse(executed_date, '%m/%d/%Y')) AS INT) AS year,    
    count(*) AS eviction_count_x_lat_long
FROM ads508_t8.evictions
WHERE latitude != ''
GROUP BY latitude, longitude, YEAR(date_parse(executed_date, '%m/%d/%Y'))
ORDER BY count(*) DESC
) AS evi
ON evi.latitude >= ceb.min_lat
    AND evi.latitude <= ceb.max_lat
    AND evi.longitude >= ceb.min_long
    AND evi.longitude <= ceb.max_long
GROUP BY ceb.blockCode, evi.year
ORDER BY ceb.blockCode, evi.year
LIMIT 50000

(4

,blockCode,year,annual_evictions_x_census_tract
0,34003013001,2017,9
1,34003013001,2018,11
2,34003013001,2019,6
3,34003013001,2021,1
4,34003013001,2022,6
5,34003013001,2023,1
6,34003016000,2017,14
7,34003016000,2018,13
8,34003016000,2019,16
9,34003016000,2020,2


In [45]:
ceb_select_to_join_stmnt01 = f"""
SELECT
    substr(blockCode,1,11) AS blockCode,
    count(*),
    min(latitude) AS min_lat,
    max(latitude) AS max_lat,
    min(longitude) AS min_long,
    max(longitude) AS max_long     
FROM {database_name}.{ceb_tsv_tbl_name}
GROUP BY substr(blockCode,1,11)
ORDER BY count(*) DESC
"""

print(ceb_select_to_join_stmnt01)

ceb_select_to_join_df01 = pd.read_sql(ceb_select_to_join_stmnt01,
                                      conn)
print(ceb_select_to_join_df01.shape)
display(ceb_select_to_join_df01.head(15))


SELECT
    substr(blockCode,1,11) AS blockCode,
    count(*),
    min(latitude) AS min_lat,
    max(latitude) AS max_lat,
    min(longitude) AS min_long,
    max(longitude) AS max_long     
FROM ads508_t8.census_block
GROUP BY substr(blockCode,1,11)
ORDER BY count(*) DESC

(2995, 6)


,blockCode,_col1,min_lat,max_lat,min_long,max_long
0,36081990100,1816,40.491307,40.584020,-74.039397,-73.757638
1,36085990100,1198,40.480000,40.604372,-74.257839,-74.036231
2,34025990000,917,40.480000,40.525226,-74.093216,-73.887437
3,36059990400,690,40.534271,40.579497,-73.767136,-73.650000
4,36059301000,412,40.819196,40.877990,-73.751307,-73.653166
5,36081107202,366,40.586281,40.645075,-73.852613,-73.767136
6,36047070203,327,40.579497,40.642814,-73.890603,-73.833618
7,34017012700,305,40.712915,40.776231,-74.143869,-74.077387
8,34013980200,297,40.674472,40.715176,-74.200854,-74.115377
9,36081071600,286,40.622462,40.663166,-73.830452,-73.748141


### Select statements to prepare for full join - `Evictions` table

In [51]:
# Display full Evictions table for review
evi_full_select_stmnt01 = f"""
    SELECT
        *
    FROM {database_name}.{evi_tsv_tbl_name}
    WHERE executed_date <> ''
    LIMIT 100000
    """

# Display SQL statement
print(evi_full_select_stmnt01)

# Run SQL statement against Athena table
evi_full_select_df01 = pd.read_sql(evi_full_select_stmnt01,
                           conn)
# Display results
print(evi_full_select_df01.shape)
display(evi_full_select_df01.head(11))


    SELECT
        *
    FROM ads508_t8.evictions
    WHERE executed_date <> ''
    LIMIT 100000
    
(72488, 20)


,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,marshal_first_name,marshal_last_name,residential_or_commercial,borough,eviction_postcode,ejectment,eviction_or_legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,56037/17,339568,547 EAST 168TH STREET,3H,02/26/2018,Thomas,Bia,Residential,BRONX,10456,Not an Ejectment,Possession,40.830857,-73.905191,3,16,145,2004227,2026100065,Claremont-Bathgate
1,B047517/19,409031,4014 CARPENTER AVENUE,4B,11/16/2022,Richard,McCoy,Residential,BRONX,10466,Not an Ejectment,Possession,40.889878,-73.862686,12,12,408,2063060,2048280031,Williamsbridge-Olinville
2,15068/17,334442,655 EAST 224TH STREET,1,09/29/2017,Thomas,Bia,Residential,BRONX,10467,Not an Ejectment,Possession,40.887599,-73.862391,12,12,394,2062985,2048260028,Williamsbridge-Olinville
3,58273/18,025388,1551 DEAN STREET,1ST FLOOR,07/12/2018,Gary,Rose,Residential,BROOKLYN,11213,Not an Ejectment,Possession,40.676166,-73.936661,8,36,311,3388499,3013400049,Crown Heights North
4,14866/19A,097278,718 PENFIELD STREET,2-F,10/24/2019,Justin,Grossman,Residential,BRONX,10470,Not an Ejectment,Possession,40.904888,-73.849089,12,11,442,2071873,2051130039,Woodlawn-Wakefield
5,66703/18BX,090391,2032 EAST 177TH ST A /K/A 2032 CROSS BRONX EXP...,1E,07/30/2019,Justin,Grossman,Residential,BRONX,10472,Not an Ejectment,Possession,40.831685,-73.856168,9,18,78,2026230,2038030019,Westchester-Unionport
6,98925/17,075402,175 WOODRUFF AVENUE,GARDEN APARTMENT,06/01/2018,Justin,Grossman,Residential,BROOKLYN,11226,Not an Ejectment,Possession,40.654641,-73.960291,14,40,50803,3115933,3050540052,Flatbush
7,304057/20,107717,555 TENTH AVENUE,32I,04/18/2022,Justin,Grossman,Residential,MANHATTAN,10018,Not an Ejectment,Possession,40.758888,-73.996022,4,3,117,1089722,1010697501,Hudson Yards-Chelsea-Flatiron-Union Square
8,210706/18,085502,2201 FIRST AVENUE,05B,03/14/2019,Henry,Daley,Residential,MANHATTAN,10029,Not an Ejectment,Possession,40.794176,-73.936754,11,8,180,1081091,1016840001,East Harlem North
9,B806500/18,396012,281 EAST 143RD STREET,07A,01/17/2019,Richard,McCoy,Residential,BRONX,10451,Not an Ejectment,Possession,40.814845,-73.924083,1,8,51,2091116,2023240001,Mott Haven-Port Morris


In [21]:
# Aggregate Evictions table based on borough and event year
evi_borough_year_stmnt01 = f"""
    SELECT
        borough,
        CAST(YEAR(date_parse(executed_date, '%m/%d/%Y')) AS INT) AS year,
        count(*) AS annual_evictions_x_borough
    FROM {database_name}.{evi_tsv_tbl_name}
    WHERE executed_date <> ''
    GROUP BY borough, YEAR(date_parse(executed_date, '%m/%d/%Y'))
    ORDER BY borough, YEAR(date_parse(executed_date, '%m/%d/%Y'))
    LIMIT 10000
    """

# Display SQL statement
print(evi_borough_year_stmnt01)

# Run SQL statement against Athena table
evi_df01_s06_borough = pd.read_sql(evi_borough_year_stmnt01,
                                   conn)
# Display results
print(evi_df01_s06_borough.shape)
display(evi_df01_s06_borough.head(35))

evi_df01_s07_borough = evi_df01_s06_borough.pivot_table(index = 'borough',
                                                        columns = 'year',
                                                        values = 'annual_evictions_x_borough',
                                                        aggfunc = 'sum',
                                                        fill_value = 0)
print(evi_df01_s07_borough.shape)
display(evi_df01_s07_borough.head(35))


    SELECT
        borough,
        CAST(YEAR(date_parse(executed_date, '%m/%d/%Y')) AS INT) AS year,
        count(*) AS annual_eviction_counts
    FROM ads508_t8.evictions
    WHERE executed_date <> ''
    GROUP BY borough, YEAR(date_parse(executed_date, '%m/%d/%Y'))
    ORDER BY borough, YEAR(date_parse(executed_date, '%m/%d/%Y'))
    LIMIT 10000
    
(35, 3)


,borough,year,annual_eviction_counts
0,BRONX,2017,7658
1,BRONX,2018,7140
2,BRONX,2019,6244
3,BRONX,2020,1088
4,BRONX,2021,29
5,BRONX,2022,1174
6,BRONX,2023,186
7,BROOKLYN,2017,6355
8,BROOKLYN,2018,6157
9,BROOKLYN,2019,5312


(5, 7)


year,2017,2018,2019,2020,2021,2022,2023
borough,,,,,,,
BRONX,7658,7140,6244,1088,29,1174,186
BROOKLYN,6355,6157,5312,1005,100,1864,273
MANHATTAN,3450,3390,2818,521,68,930,144
QUEENS,4325,4452,3705,696,36,811,57
STATEN ISLAND,734,691,636,112,35,271,21


In [ ]:
evi_ceb_join_select_stmnt01 = f"""
SELECT
    ceb.blockCode,
    evi.year,
    sum(evi.eviction_count_x_lat_long) AS annual_evictions_x_census_tract
FROM (SELECT
    substr(blockCode,1,11) AS blockCode,
    count(*),
    min(latitude) AS min_lat,
    max(latitude) AS max_lat,
    min(longitude) AS min_long,
    max(longitude) AS max_long     
FROM {database_name}.{ceb_tsv_tbl_name}
GROUP BY substr(blockCode,1,11)
ORDER BY count(*) DESC) AS ceb
INNER JOIN (
SELECT
    CAST(latitude AS DOUBLE) AS latitude,
    CAST(longitude AS DOUBLE) AS longitude,
    CAST(YEAR(date_parse(executed_date, '%m/%d/%Y')) AS INT) AS year,    
    count(*) AS eviction_count_x_lat_long
FROM {database_name}.{evi_tsv_tbl_name}
WHERE latitude != ''
GROUP BY latitude, longitude, YEAR(date_parse(executed_date, '%m/%d/%Y'))
ORDER BY count(*) DESC
) AS evi
ON evi.latitude >= ceb.min_lat
    AND evi.latitude <= ceb.max_lat
    AND evi.longitude >= ceb.min_long
    AND evi.longitude <= ceb.max_long
GROUP BY ceb.blockCode, evi.year
ORDER BY ceb.blockCode, evi.year
LIMIT 50000
"""

print(evi_ceb_join_select_stmnt01)

evi_ceb_join_select_df01 = pd.read_sql(evi_ceb_join_select_stmnt01,
                           conn)
print(evi_ceb_join_select_df01.shape)
display(evi_ceb_join_select_df01.head(15))

In [22]:
# Run query to review a sample of records
cri_full_select_stmnt01 = f"""
    SELECT
        *
    FROM {database_name}.{cri_pqt_tbl_name}
    TABLESAMPLE BERNOULLI(2)
    LIMIT 10000
    """

# Display SQL statement
print(cri_full_select_stmnt01)

# Run SQL statement against Athena table
cri_full_select_df01 = pd.read_sql(cri_full_select_stmnt01,
                                   conn)
# Display results
print(cri_full_select_df01.shape)
display(cri_full_select_df01.head(5))


    SELECT
        *
    FROM ads508_t8.crime_pqt
    TABLESAMPLE BERNOULLI(2)
    LIMIT 10000
    
(10000, 35)


,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,addr_pct_cd,rpt_dt,ky_cd,ofns_desc,pd_cd,...,latitude,longitude,lat_lon,patrol_boro,station_name,vic_age_group,vic_race,vic_sex,law_cat_cd,borough
0,840372734,06/28/2007,19:30:00,06/28/2007,19:45:00,60,06/28/2007,360,,661,...,40.5761573,-73.9759838,"(40.5761573, -73.9759838)",PATROL BORO BKLYN SOUTH,,25-44,BLACK,M,MISDEMEANOR,None
1,436000847,06/16/2007,13:35:00,,,103,06/16/2007,344,ASSAULT 3 & RELATED OFFENSES,101,...,40.707047475,-73.792611904,"(40.707047475, -73.792611904)",PATROL BORO QUEENS SOUTH,,18-24,ASIAN / PACIFIC ISLANDER,F,MISDEMEANOR,None
2,835383582,02/19/2009,16:58:00,02/19/2009,16:58:00,122,02/19/2009,341,PETIT LARCENY,338,...,40.581537433,-74.1117436,"(40.581537433, -74.1117436)",PATROL BORO STATEN ISLAND,,,UNKNOWN,D,MISDEMEANOR,None
3,331920796,11/18/2015,23:15:00,,,52,11/18/2015,101,MURDER & NON-NEGL. MANSLAUGHTER,,...,40.864694331,-73.899576908,"(40.864694331, -73.899576908)",,,18-24,WHITE HISPANIC,M,FELONY,None
4,712999455,07/12/2020,00:45:00,,,61,07/12/2020,101,MURDER & NON-NEGL. MANSLAUGHTER,,...,40.58615896100008,-73.93816979699994,"(40.58615896100008, -73.93816979699994)",,,18-24,BLACK,M,FELONY,None


In [44]:
# Aggregate Evictions table based on borough and event year
cri_borough_year_type_stmnt01 = f"""
    SELECT
        borough,
        concat(cast(YEAR(date_parse(cmplnt_fr_dt, '%m/%d/%Y')) AS varchar), ' - ', law_cat_cd) AS year_w_complaint,
        count(*) AS annual_complaint_counts
    FROM {database_name}.{cri_pqt_tbl_name}
    TABLESAMPLE BERNOULLI(2)
    WHERE cmplnt_fr_dt <> ''
        AND YEAR(date_parse(cmplnt_fr_dt, '%m/%d/%Y')) > 2003
    GROUP BY borough, concat(cast(YEAR(date_parse(cmplnt_fr_dt, '%m/%d/%Y')) AS varchar), ' - ', law_cat_cd)
    ORDER BY borough, concat(cast(YEAR(date_parse(cmplnt_fr_dt, '%m/%d/%Y')) AS varchar), ' - ', law_cat_cd)
    LIMIT 1000
    """

# Display SQL statement
print(cri_borough_year_type_stmnt01)

# Run SQL statement against Athena table
cri_borough_year_type_df01 = pd.read_sql(cri_borough_year_type_stmnt01,
                                   conn)
# Display results
print(cri_borough_year_type_df01.shape)
display(cri_borough_year_type_df01.head(35))

cri_borough_year_type_df02 = cri_borough_year_type_df01.pivot_table(index = 'borough',
                                                        columns = 'year_w_complaint',
                                                        values = 'annual_complaint_counts',
                                                        aggfunc = 'sum',
                                                        fill_value = 0)
print(cri_borough_year_type_df02.shape)
display(cri_borough_year_type_df02.head(35))


    SELECT
        borough,
        concat(cast(YEAR(date_parse(cmplnt_fr_dt, '%m/%d/%Y')) AS varchar), ' - ', law_cat_cd) AS year_w_complaint,
        count(*) AS annual_complaint_counts
    FROM ads508_t8.crime_pqt
    TABLESAMPLE BERNOULLI(2)
    WHERE cmplnt_fr_dt <> ''
        AND YEAR(date_parse(cmplnt_fr_dt, '%m/%d/%Y')) > 2003
    GROUP BY borough, concat(cast(YEAR(date_parse(cmplnt_fr_dt, '%m/%d/%Y')) AS varchar), ' - ', law_cat_cd)
    ORDER BY borough, concat(cast(YEAR(date_parse(cmplnt_fr_dt, '%m/%d/%Y')) AS varchar), ' - ', law_cat_cd)
    LIMIT 1000
    
(289, 3)


,borough,year_w_complaint,annual_complaint_counts
0,BRONX,2004 - FELONY,6
1,BRONX,2004 - MISDEMEANOR,4
2,BRONX,2005 - FELONY,20
3,BRONX,2005 - MISDEMEANOR,18
4,BRONX,2005 - VIOLATION,1
5,BRONX,2006 - FELONY,618
6,BRONX,2006 - MISDEMEANOR,1280
7,BRONX,2006 - VIOLATION,307
8,BRONX,2007 - FELONY,654
9,BRONX,2007 - MISDEMEANOR,1452


(5, 54)


year_w_complaint,2004 - FELONY,2004 - MISDEMEANOR,2004 - VIOLATION,2005 - FELONY,2005 - MISDEMEANOR,2005 - VIOLATION,2006 - FELONY,2006 - MISDEMEANOR,2006 - VIOLATION,2007 - FELONY,...,2018 - VIOLATION,2019 - FELONY,2019 - MISDEMEANOR,2019 - VIOLATION,2020 - FELONY,2020 - MISDEMEANOR,2020 - VIOLATION,2021 - FELONY,2021 - MISDEMEANOR,2021 - VIOLATION
borough,,,,,,,,,,,,,,,,,,,,,
BRONX,6,4,0,20,18,1,618,1280,307,654,...,328,545,1073,382,555,915,311,599,875,347
BROOKLYN,5,2,0,35,35,5,1047,1694,365,971,...,438,892,1292,376,762,1145,417,848,1118,411
MANHATTAN,9,3,0,32,23,3,884,1372,268,890,...,297,688,1312,291,620,998,230,725,1125,305
QUEENS,9,0,1,21,19,4,706,1131,243,690,...,306,556,994,314,548,859,302,618,993,320
STATEN ISLAND,3,0,0,7,7,5,116,337,106,114,...,78,79,201,74,86,157,63,99,190,91


In [39]:
# Run query to review a sample of records
grd_full_select_stmnt01 = f"""
    SELECT
        grd.school_name,
        grd.cohort,
        grd.total_grads_n,
        grd.dropped_out_n,
        hsi.borough
    FROM {database_name}.{grd_tsv_tbl_name} AS grd
    LEFT JOIN {database_name}.{hsi_tsv_tbl_name} AS hsi
    ON grd.dbn = hsi.dbn
    WHERE total_grads_n <> 's'
    LIMIT 100000
    """

# Display SQL statement
print(grd_full_select_stmnt01)

# Run SQL statement against Athena table
grd_full_select_df01 = pd.read_sql(grd_full_select_stmnt01,
                                   conn)
# Display results
print(grd_full_select_df01.shape)
display(grd_full_select_df01.head(7))


    SELECT
        grd.school_name,
        grd.cohort,
        grd.total_grads_n,
        grd.dropped_out_n,
        hsi.borough
    FROM ads508_t8.grad_outcomes AS grd
    LEFT JOIN ads508_t8.hs_info AS hsi
    ON grd.dbn = hsi.dbn
    WHERE total_grads_n <> 's'
    LIMIT 100000
    
(16704, 5)


,school_name,cohort,total_grads_n,dropped_out_n,borough
0,HENRY STREET SCHOOL FOR INTERNATIONAL,2004,37,3,Manhattan
1,HENRY STREET SCHOOL FOR INTERNATIONAL,2005,43,9,Manhattan
2,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,43,11,Manhattan
3,HENRY STREET SCHOOL FOR INTERNATIONAL,2006 Aug,44,11,Manhattan
4,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2001,46,6,Manhattan
5,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2002,33,1,Manhattan
6,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2003,67,11,Manhattan


In [43]:
# Run query to review a sample of records
grd_select_borough_stmnt01 = f"""
    SELECT
        hsi.borough,
        grd.cohort,
        sum(CAST(grd.total_grads_n AS INT) AS annual_grad_n,
        sum(CAST(grd.dropped_out_n AS INT) AS annual_dropped_out_n
    FROM {database_name}.{grd_tsv_tbl_name} AS grd
    LEFT JOIN {database_name}.{hsi_tsv_tbl_name} AS hsi
    ON grd.dbn = hsi.dbn
    WHERE total_grads_n <> 's'
    GROUP BY hsi.borough, grd.cohort
    LIMIT 100000
    """

# Display SQL statement
print(grd_select_borough_stmnt01)

# Run SQL statement against Athena table
grd_select_borough_stmnt01_df01 = pd.read_sql(grd_select_borough_stmnt01,
                                   conn)
# Display results
print(grd_select_borough_stmnt01_df01.shape)
display(grd_select_borough_stmnt01_df01.head(7))

Failed to execute query.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/pyathena/common.py", line 305, in _execute
    **request
  File "/opt/conda/lib/python3.7/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 379, in __call__
    do = self.iter(retry_state=retry_state)
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 314, in iter
    return fut.result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 428, in result
    return self.__get_result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 384, in __get_result
    raise self._exception
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 382, in __call__
    result = fn(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 530, in _api_call
    return


    SELECT
        hsi.borough,
        grd.cohort,
        sum(CAST(grd.total_grads_n AS INT) AS annual_grad_n,
        sum(CAST(grd.dropped_out_n AS INT) AS annual_dropped_out_n
    FROM ads508_t8.grad_outcomes AS grd
    LEFT JOIN ads508_t8.hs_info AS hsi
    ON grd.dbn = hsi.dbn
    WHERE total_grads_n <> 's'
    GROUP BY hsi.borough, grd.cohort
    LIMIT 100000
    


DatabaseError: Execution failed on sql: 
    SELECT
        hsi.borough,
        grd.cohort,
        sum(CAST(grd.total_grads_n AS INT) AS annual_grad_n,
        sum(CAST(grd.dropped_out_n AS INT) AS annual_dropped_out_n
    FROM ads508_t8.grad_outcomes AS grd
    LEFT JOIN ads508_t8.hs_info AS hsi
    ON grd.dbn = hsi.dbn
    WHERE total_grads_n <> 's'
    GROUP BY hsi.borough, grd.cohort
    LIMIT 100000
    
An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 4:44: mismatched input 'AS'. Expecting: ',', <expression>
unable to rollback

# Show the Tables

In [ ]:
show_tbl_stmnt = f"SHOW TABLES in {database_name}"

In [ ]:
df_tables = pd.read_sql(show_tbl_stmnt,
                        conn)
df_tables.head(17)

if abt_tbl_name in df_tables.values:
    ingest_create_athena_table_parquet_passed = True

%store ingest_create_athena_table_parquet_passed

# Run Sample Query

abt_borough01 = 'bronx'

abt_select_borough_stmnt02 = f"""
    SELECT * FROM {database_name}.{abt_tbl_name}
    WHERE blockcode IS NOT NULL
    LIMIT 10000
    """

print(abt_select_borough_stmnt02)

abt_df02_s01 = pd.read_sql(abt_select_borough_stmnt02,
                           conn)

print(abt_df02_s01.shape)
display(abt_df02_s01.head(17))

path_balanced = "./data-clarify/amazon_reviews_us_giftcards_software_videogames_balanced.csv"
df_balanced.to_csv(path_balanced, index=False, header=True)

if not abt_df02_s01.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN CONVERTED TO PARQUET. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

## Review the New Athena Table in the Glue Catalog

In [ ]:
display(
    HTML(
        f'<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={region}#">AWS Glue Catalog</a></b>'
    )
)

## Store Variables for the Next Notebooks

In [ ]:
%store

## Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}